In [1]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [0]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import tensorflow_datasets as tfds
import json
import unicodedata

In [3]:
!mkdir -p ~/.kaggle
token = {"username":"seunghwan1228","key":"0f6351758ad29d87352cba99de319590"}
with open('/content/drive/My Drive/Colab Notebooks/Kaggle Kernel/kaggle.json', 'w') as file:
  json.dump(token, file)  
!cp drive/'My Drive'/'Colab Notebooks'/'Kaggle Kernel'/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!ls ~/.kaggle
!ls -l ~/.kaggle
!cat ~/.kaggle/kaggle.json
!kaggle competitions download -c jigsaw-unintended-bias-in-toxicity-classification

kaggle.json
total 4
-rw------- 1 root root 72 Mar 19 13:20 kaggle.json
{"username": "seunghwan1228", "key": "0f6351758ad29d87352cba99de319590"}Warning: Looks like you're using an outdated API Version, please consider updating (server 1.5.6 / client 1.5.4)
 41% 5.00M/12.3M [00:00<00:00, 10.9MB/s]
100% 12.3M/12.3M [00:00<00:00, 25.0MB/s]
 57% 9.00M/15.8M [00:00<00:00, 21.3MB/s]
100% 15.8M/15.8M [00:00<00:00, 35.5MB/s]
  0% 0.00/224k [00:00<?, ?B/s]
100% 224k/224k [00:00<00:00, 69.6MB/s]
 99% 273M/276M [00:05<00:00, 44.7MB/s]
100% 276M/276M [00:05<00:00, 52.4MB/s]
 74% 9.00M/12.1M [00:01<00:00, 6.51MB/s]
100% 12.1M/12.1M [00:01<00:00, 8.12MB/s]
 94% 61.0M/64.7M [00:01<00:00, 31.1MB/s]
100% 64.7M/64.7M [00:01<00:00, 53.5MB/s]
 32% 5.00M/15.9M [00:00<00:00, 11.8MB/s]
100% 15.9M/15.9M [00:00<00:00, 29.4MB/s]


In [4]:
!unzip /content/train.csv.zip

Archive:  /content/train.csv.zip
  inflating: train.csv               


In [0]:
df = pd.read_csv('/content/train.csv')

In [6]:
df.head()

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,bisexual,black,buddhist,christian,female,heterosexual,hindu,homosexual_gay_or_lesbian,intellectual_or_learning_disability,jewish,latino,male,muslim,other_disability,other_gender,other_race_or_ethnicity,other_religion,other_sexual_orientation,physical_disability,psychiatric_or_mental_illness,transgender,white,created_date,publication_id,parent_id,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
0,59848,0.000000,"This is so cool. It's like, 'would you want yo...",0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:41.987077+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
1,59849,0.000000,Thank you!! This would make my life a lot less...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:42.870083+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
2,59852,0.000000,This is such an urgent design problem; kudos t...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:45.222647+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
3,59855,0.000000,Is this something I'll be able to install on m...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:47.601894+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
4,59856,0.893617,haha you guys are a bunch of losers.,0.021277,0.0,0.021277,0.87234,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2015-09-29 10:50:48.488476+00,2,NaN,2006,rejected,0,0,0,1,0,0.0,4,47


In [0]:
df = df[['comment_text', 'target']]

In [8]:
df.head()

,comment_text,target
0,"This is so cool. It's like, 'would you want yo...",0.000000
1,Thank you!! This would make my life a lot less...,0.000000
2,This is such an urgent design problem; kudos t...,0.000000
3,Is this something I'll be able to install on m...,0.000000
4,haha you guys are a bunch of losers.,0.893617


In [0]:
def convert_label(x):
  if x >= 0.5:
    return 1
  else:
    return 0

In [0]:
df['target_category'] = df['target'].apply(convert_label)

In [11]:
df.head()

,comment_text,target,target_category
0,"This is so cool. It's like, 'would you want yo...",0.000000,0
1,Thank you!! This would make my life a lot less...,0.000000,0
2,This is such an urgent design problem; kudos t...,0.000000,0
3,Is this something I'll be able to install on m...,0.000000,0
4,haha you guys are a bunch of losers.,0.893617,1


In [12]:
df.loc[0, 'comment_text']

"This is so cool. It's like, 'would you want your mother to read this??' Really great idea, well done!"

In [0]:
tokenizer = tfds.features.text.SubwordTextEncoder.build_from_corpus(df['comment_text'].tolist(), target_vocab_size=2**13)

In [14]:
tokenizer.vocab_size

8216

In [15]:
tokenizer.encode(df.loc[0, 'comment_text'])

[109,
 9,
 66,
 4424,
 8,
 144,
 7999,
 7,
 625,
 2,
 7999,
 45,
 16,
 138,
 46,
 3676,
 3,
 318,
 264,
 3312,
 151,
 2038,
 7992,
 329,
 1523,
 2,
 223,
 1176,
 7993]

In [0]:
def tokenize_text(text_list):
  result = []
  for text in text_list:
    convert_token = tokenizer.encode(text)
    result.append(convert_token)
  
  return result

In [0]:
tokenized_result = tokenize_text(df['comment_text'].tolist())

In [0]:
def check_max_length(x):
  length_result = []
  for i in x:
    length = len(i)
    length_result.append(length)
  
  length_result = np.array(length_result)
  return np.max(length_result)

In [20]:
check_max_length(tokenized_result)

1659

In [0]:
def pad_text_data(text_list):
  return tf.keras.preprocessing.sequence.pad_sequences(text_list, maxlen=check_max_length(tokenized_result), padding='post')

In [0]:
text_data = pad_text_data(tokenized_result)

In [0]:
label_data = df['target_category'].values

Define Model


In [0]:
class Attention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(Attention, self).__init__()
    self.units = units
    self.h = tf.keras.layers.Dense(units)
    self.u = tf.keras.layers.Dense(units, use_bias=False)

  def call(self, inputs):
    u_it = tf.nn.tanh(self.h(inputs))  # b, n, units
    # print(u_it)

    uw = self.u(u_it)
    a_it = tf.keras.layers.Activation('softmax')(uw)

    si = a_it * inputs
    si = tf.reduce_sum(si, axis=1)
    return si

In [0]:
input_ = tf.keras.layers.Input(shape=(1659,))
emb = tf.keras.layers.Embedding(tokenizer.vocab_size+1, 300, mask_zero=True)(input_)
bi_gru = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(128, return_sequences=True))(emb)
att = Attention(128*2)(bi_gru)
dense = tf.keras.layers.Dense(128, activation='relu')(att)
out_ = tf.keras.layers.Dense(1, activation='sigmoid')(dense)

model = tf.keras.models.Model(input_, out_)

In [0]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [0]:
hist = model.fit(text_data, label_data, batch_size=64, epochs=50)

Train on 1804874 samples
Epoch 1/50
1804874/1804874 [==============================] - 3827s 2ms/sample - loss: 0.1430 - accuracy: 0.9466
Epoch 2/50
 756288/1804874 [===========>..................] - ETA: 36:39 - loss: 0.1286 - accuracy: 0.9506